In [1]:
import pandas as pd

# Loading Datasets

In [6]:
#Loads files from github
covid_url ='https://raw.githubusercontent.com/zhrmnch/CSI4142_Phase3/main/datasets/COVID19_cases.csv'
MCI_url ='https://raw.githubusercontent.com/zhrmnch/CSI4142_Phase3/main/datasets/Major_Crime_Indicators.csv'
neighbourhood_crime_rate_url ='https://raw.githubusercontent.com/zhrmnch/CSI4142_Phase3/main/datasets/Neighbourhood_Crime_Rates.csv'

# Prepare Crime fact table

In [7]:
# MC Dataset
crimedf = pd.read_csv(MCI_url)
crimedf = crimedf.drop_duplicates()
crimedf['occurrencedate'] = pd.to_datetime(crimedf['occurrencedate'])
crimedf['reporteddate'] = pd.to_datetime(crimedf['reporteddate'])
crimedf = crimedf.drop(columns=["event_unique_id","Index_","X","Y","ObjectId","Hood_ID","reportedyear","reportedmonth", "reportedday", "reporteddayofyear", "reporteddayofweek", "reportedhour","occurrenceyear", "occurrencemonth", "occurrenceday", "occurrencedayofyear", "occurrencedayofweek", "occurrencehour"])

# Manual changes to the neighbourhood column to account for typos
crimedf['Neighbourhood'] = crimedf['Neighbourhood'].replace('LAmoreaux', "L'Amoreaux")
crimedf['Neighbourhood'] = crimedf['Neighbourhood'].replace('OConnor-Parkview', "O'Connor-Parkview")
crimedf['Neighbourhood'] = crimedf['Neighbourhood'].replace('Mimico', "Mimico (includes Humber Bay Shores)")
crimedf['Neighbourhood'] = crimedf['Neighbourhood'].replace('Tam OShanter-Sullivan', "Tam O'Shanter-Sullivan")

# Remove crimes with no location
crimedf = crimedf[(crimedf["Neighbourhood"] != "NSA") & (crimedf["Latitude"] != 0) & (crimedf["Longitude"] != 0)]
crimedf = crimedf.drop_duplicates()
crimedf

#crimedf.describe(include = ['object','int'])


,Division,occurrencedate,reporteddate,location_type,premises_type,ucr_code,ucr_ext,offence,mci_category,Neighbourhood,Longitude,Latitude
0,D31,2014-01-03 05:00:00+00:00,2014-01-03 05:00:00+00:00,"Apartment (Rooming House, Condo)",Apartment,1430,100,Assault,Assault,York University Heights,-79.504475,43.766371
1,D42,2014-01-03 05:00:00+00:00,2014-01-03 05:00:00+00:00,"Single Home, House (Attach Garage, Cottage, Mo...",House,2120,200,B&E,Break and Enter,Malvern,-79.217083,43.814861
2,D22,2014-01-03 05:00:00+00:00,2014-01-03 05:00:00+00:00,"Open Areas (Lakes, Parks, Rivers)",Outside,1430,100,Assault,Assault,Long Branch,-79.534957,43.589085
3,D53,2014-01-03 05:00:00+00:00,2014-01-03 05:00:00+00:00,Other Commercial / Corporate Places (For Profi...,Commercial,2130,210,Theft Over,Theft Over,Thorncliffe Park,-79.346588,43.703212
4,D22,2014-01-03 05:00:00+00:00,2014-01-03 05:00:00+00:00,Convenience Stores,Commercial,1610,210,Robbery - Business,Robbery,Islington-City Centre West,-79.531916,43.645247
...,...,...,...,...,...,...,...,...,...,...,...,...
301227,D55,2022-06-28 04:00:00+00:00,2022-06-29 04:00:00+00:00,"Single Home, House (Attach Garage, Cottage, Mo...",House,2135,210,Theft Of Motor Vehicle,Auto Theft,Woodbine-Lumsden,-79.308515,43.695118
301228,D32,2022-06-28 04:00:00+00:00,2022-06-29 04:00:00+00:00,"Single Home, House (Attach Garage, Cottage, Mo...",House,2135,210,Theft Of Motor Vehicle,Auto Theft,Westminster-Branson,-79.455098,43.790363
301229,D43,2022-06-28 04:00:00+00:00,2022-06-29 04:00:00+00:00,"Parking Lots (Apt., Commercial Or Non-Commercial)",Outside,2135,210,Theft Of Motor Vehicle,Auto Theft,Woburn,-79.234954,43.775740
301230,D41,2022-05-20 04:00:00+00:00,2022-06-29 04:00:00+00:00,Other Commercial / Corporate Places (For Profi...,Commercial,2135,210,Theft Of Motor Vehicle,Auto Theft,Dorset Park,-79.281037,43.765801


# Prepare COVID Dataset

In [8]:
# COVID Dataset
covdf = pd.read_csv(covid_url)
covdf['Reported Date'] = pd.to_datetime(covdf['Reported Date'])
covdf['Episode Date'] = pd.to_datetime(covdf['Episode Date'])

In [9]:
covdf.drop_duplicates()

,_id,Assigned_ID,Outbreak Associated,Age Group,Neighbourhood Name,FSA,Source of Infection,Classification,Episode Date,Reported Date,Client Gender,Outcome,Ever Hospitalized,Ever in ICU,Ever Intubated
0,1,1,Sporadic,50 to 59 Years,Willowdale East,M2N,Travel,CONFIRMED,2020-01-22,2020-01-23,FEMALE,RESOLVED,No,No,No
1,2,2,Sporadic,50 to 59 Years,Willowdale East,M2N,Travel,CONFIRMED,2020-01-21,2020-01-23,MALE,RESOLVED,Yes,No,No
2,3,3,Sporadic,20 to 29 Years,Parkwoods-Donalda,M3A,Travel,CONFIRMED,2020-02-05,2020-02-21,FEMALE,RESOLVED,No,No,No
3,4,4,Sporadic,60 to 69 Years,Church-Yonge Corridor,M4W,Travel,CONFIRMED,2020-02-16,2020-02-25,FEMALE,RESOLVED,No,No,No
4,5,5,Sporadic,60 to 69 Years,Church-Yonge Corridor,M4W,Travel,CONFIRMED,2020-02-20,2020-02-26,MALE,RESOLVED,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390550,390551,401695,Sporadic,40 to 49 Years,Newtonbrook West,M2M,No Information,CONFIRMED,2023-01-30,2023-01-31,FEMALE,ACTIVE,No,No,No
390551,390552,401696,Sporadic,50 to 59 Years,Westminster-Branson,M2R,No Information,CONFIRMED,2023-01-30,2023-01-31,FEMALE,ACTIVE,No,No,No
390552,390553,401697,Sporadic,50 to 59 Years,Banbury-Don Mills,M3B,No Information,CONFIRMED,2023-01-30,2023-01-31,MALE,ACTIVE,No,No,No
390553,390554,401698,Outbreak Associated,90 and older,Beechborough-Greenbrook,M6M,"Outbreaks, Healthcare Institutions",CONFIRMED,2023-01-30,2023-01-31,FEMALE,ACTIVE,No,No,No


# Prepare Neighbourhood Table

In [11]:
# Neighbourhood Table - DONE
neighbourhooddf = pd.DataFrame()
neighbourhooddf = pd.read_csv(neighbourhood_crime_rate_url)
neighbourhooddf = neighbourhooddf.loc[:, ['Neighbourhood','Shape__Area','Shape__Length']]
neighbourhooddf = neighbourhooddf.drop_duplicates() # remove dups
neighbourhooddf['neighbourhood_PK'] = range(1, len(neighbourhooddf) + 1) # generate surr key
neighbourhooddf = neighbourhooddf[['neighbourhood_PK'] + [col for col in neighbourhooddf.columns if col != 'neighbourhood_PK']]

# Manual changes to the neighbourhood column
neighbourhooddf['Neighbourhood'] = neighbourhooddf['Neighbourhood'].replace('Mimico (includes Humber Bay Shores) ', 'Mimico (includes Humber Bay Shores)')
neighbourhooddf

,neighbourhood_PK,Neighbourhood,Shape__Area,Shape__Length
0,1,Yonge-St.Clair,1.161315e+06,5873.270507
1,2,York University Heights,1.324666e+07,18504.777616
2,3,Lansing-Westgate,5.346186e+06,11112.109419
3,4,Yorkdale-Glen Park,6.038326e+06,10079.426837
4,5,Stonegate-Queensway,7.946202e+06,11853.189803
...,...,...,...,...
135,136,Pleasant View,2.967493e+06,7944.829782
136,137,Wychwood,1.682111e+06,5435.022062
137,138,Leaside-Bennington,4.784766e+06,14836.618542
138,139,Briar Hill-Belgravia,1.830325e+06,6289.707250


# Prepare Location Table

In [12]:
# Location Table - DONE
locationdf = pd.DataFrame()
locationdf = crimedf[['Division', 'Longitude', 'Latitude']].copy()
locationdf = locationdf.drop_duplicates() # remove dups
locationdf['location_PK'] = range(1, len(locationdf) + 1) # generate surr key
locationdf = locationdf[['location_PK'] + [col for col in locationdf.columns if col != 'location_PK']]
locationdf

,location_PK,Division,Longitude,Latitude
0,1,D31,-79.504475,43.766371
1,2,D42,-79.217083,43.814861
2,3,D22,-79.534957,43.589085
3,4,D53,-79.346588,43.703212
4,5,D22,-79.531916,43.645247
...,...,...,...,...
300968,20104,D33,-79.393966,43.794151
301202,20105,D53,-79.406298,43.702062
301211,20106,D43,-79.139115,43.785521
301221,20107,D12,-79.501044,43.675449


# Prepare Premise Table

In [13]:
# Premise Table - DONE
premisedf = pd.DataFrame()
premisedf = crimedf[['location_type','premises_type']].copy()
premisedf = premisedf.drop_duplicates() # remove dups
premisedf['premise_PK'] = range(1, len(premisedf) + 1) # generate surr key
premisedf = premisedf[['premise_PK'] + [col for col in premisedf.columns if col != 'premise_PK']]
premisedf

,premise_PK,location_type,premises_type
0,1,"Apartment (Rooming House, Condo)",Apartment
1,2,"Single Home, House (Attach Garage, Cottage, Mo...",House
2,3,"Open Areas (Lakes, Parks, Rivers)",Outside
3,4,Other Commercial / Corporate Places (For Profi...,Commercial
4,5,Convenience Stores,Commercial
7,6,"Parking Lots (Apt., Commercial Or Non-Commercial)",Outside
18,7,Unknown,Other
19,8,"Commercial Dwelling Unit (Hotel, Motel, B & B,...",Commercial
21,9,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside
22,10,Bar / Restaurant,Commercial


# Prepare Incident Table

In [15]:
# Incident Table - DONE
incidentdf = pd.DataFrame()
incidentdf = crimedf[['mci_category','offence','ucr_code', 'ucr_ext']].copy()
incidentdf = incidentdf.drop_duplicates() # remove dups
incidentdf['incident_PK'] = range(1, len(incidentdf) + 1) # generate surr key
incidentdf = incidentdf[['incident_PK'] + [col for col in incidentdf.columns if col != 'incident_PK']]
incidentdf

,incident_PK,mci_category,offence,ucr_code,ucr_ext
0,1,Assault,Assault,1430,100
1,2,Break and Enter,B&E,2120,200
3,3,Theft Over,Theft Over,2130,210
4,4,Robbery,Robbery - Business,1610,210
7,5,Theft Over,Theft From Motor Vehicle Over,2132,200
9,6,Break and Enter,B&E W'Intent,2120,220
12,7,Assault,Assault - Force/Thrt/Impede,1430,110
13,8,Assault,Assault Peace Officer,1460,100
22,9,Assault,Assault With Weapon,1420,100
24,10,Auto Theft,Theft Of Motor Vehicle,2135,210


# Prepare Occuence Table

In [17]:
# Occurrence Date Table - DONE
occurencedf = pd.DataFrame()
occurencedf = crimedf[['occurrencedate']].copy()
occurencedf = occurencedf.drop_duplicates() # remove dups
occurencedf['occurence_PK'] = range(1, len(occurencedf) + 1) # generate surr key
occurencedf = occurencedf[['occurence_PK'] + [col for col in occurencedf.columns if col != 'occurence_PK']]

# Generating new columns
occurencedf['occurrencehour'] = pd.DatetimeIndex(occurencedf['occurrencedate']).hour
occurencedf['occurrenceday'] = pd.DatetimeIndex(occurencedf['occurrencedate']).day
occurencedf['occurrencemonth'] = pd.DatetimeIndex(occurencedf['occurrencedate']).month
occurencedf['occurrenceyear'] = pd.DatetimeIndex(occurencedf['occurrencedate']).year
occurencedf

,occurence_PK,occurrencedate,occurrencehour,occurrenceday,occurrencemonth,occurrenceyear
0,1,2014-01-03 05:00:00+00:00,5,3,1,2014
5,2,2014-01-02 05:00:00+00:00,5,2,1,2014
7,3,2013-12-31 05:00:00+00:00,5,31,12,2013
15,4,2013-12-18 05:00:00+00:00,5,18,12,2013
28,5,2013-12-20 05:00:00+00:00,5,20,12,2013
...,...,...,...,...,...,...
300040,3538,2022-06-21 04:00:00+00:00,4,21,6,2022
300071,3539,2022-06-23 04:00:00+00:00,4,23,6,2022
301000,3540,2022-06-29 04:00:00+00:00,4,29,6,2022
301005,3541,2022-06-30 04:00:00+00:00,4,30,6,2022


# Prepare Reported Table

In [18]:
# Reported Date Table - DONE
reporteddf = pd.DataFrame()
reporteddf = crimedf[['reporteddate']].copy() # create location dimension
reporteddf = reporteddf.drop_duplicates() # remove dups
reporteddf['reported_PK'] = range(1, len(reporteddf) + 1) # generate surr key
reporteddf = reporteddf[['reported_PK'] + [col for col in reporteddf.columns if col != 'reported_PK']]

# Generating new columns
reporteddf['reportedhour'] = pd.DatetimeIndex(reporteddf['reporteddate']).hour
reporteddf['reportedday'] = pd.DatetimeIndex(reporteddf['reporteddate']).day
reporteddf['reportedmonth'] = pd.DatetimeIndex(reporteddf['reporteddate']).month
reporteddf['reportedyear'] = pd.DatetimeIndex(reporteddf['reporteddate']).year
reporteddf

,reported_PK,reporteddate,reportedhour,reportedday,reportedmonth,reportedyear
0,1,2014-01-03 05:00:00+00:00,5,3,1,2014
31,2,2014-01-04 05:00:00+00:00,5,4,1,2014
99,3,2014-01-05 05:00:00+00:00,5,5,1,2014
101,4,2014-01-06 05:00:00+00:00,5,6,1,2014
148,5,2014-01-07 05:00:00+00:00,5,7,1,2014
...,...,...,...,...,...,...
300038,3099,2022-06-22 04:00:00+00:00,4,22,6,2022
300040,3100,2022-06-21 04:00:00+00:00,4,21,6,2022
300056,3101,2022-06-24 04:00:00+00:00,4,24,6,2022
301000,3102,2022-06-29 04:00:00+00:00,4,29,6,2022


# Stage COVID Fact Table from COVID dataframe

In [19]:
# Monthly COVID Fact Table/Cases Table
casesdf = pd.DataFrame()
casesdf = covdf.groupby([pd.to_datetime(covdf['Episode Date']).dt.to_period('M'), 'Neighbourhood Name'])['_id'].count().reset_index()
casesdf.columns = ['Date', 'Neighbourhood', 'Cases']

# Date conversions
casesdf['date'] = casesdf['Date'].dt.to_timestamp(freq='M')
casesdf['year'] = casesdf['Date'].dt.year
casesdf['month'] = casesdf['Date'].dt.month
casesdf = casesdf.drop('Date', axis=1)

# Manual changes to typos in neighbourhood column
casesdf['Neighbourhood'] = casesdf['Neighbourhood'].replace('Cabbagetown-South St. James Town', 'Cabbagetown-South St.James Town')
casesdf['Neighbourhood'] = casesdf['Neighbourhood'].replace('Briar Hill - Belgravia', 'Briar Hill-Belgravia')
casesdf['Neighbourhood'] = casesdf['Neighbourhood'].replace('Danforth-East York', 'Danforth East York')
casesdf['Neighbourhood'] = casesdf['Neighbourhood'].replace('North St. James Town', 'North St.James Town')
casesdf

,Neighbourhood,Cases,date,year,month
0,Bay Street Corridor,1,2020-01-31,2020,1
1,Mount Dennis,1,2020-01-31,2020,1
2,Willowdale East,2,2020-01-31,2020,1
3,Church-Yonge Corridor,2,2020-02-29,2020,2
4,Don Valley Village,1,2020-02-29,2020,2
...,...,...,...,...,...
4864,Wychwood,31,2023-01-31,2023,1
4865,Yonge-Eglinton,8,2023-01-31,2023,1
4866,Yonge-St.Clair,19,2023-01-31,2023,1
4867,York University Heights,60,2023-01-31,2023,1


# Merge COVID Fact Table with Neighbourhood Table

In [20]:
# Generating Surr Key for COVID Month Table - DONE
casesdf = casesdf.merge(neighbourhooddf, on=['Neighbourhood'])
casesdf = casesdf.drop(['date', 'Neighbourhood', 'Shape__Area', 'Shape__Length'], axis=1)
casesdf

,Cases,year,month,neighbourhood_PK
0,1,2020,1,94
1,25,2020,3,94
2,14,2020,4,94
3,10,2020,5,94
4,1,2020,6,94
...,...,...,...,...
4864,27,2022,9,50
4865,30,2022,10,50
4866,9,2022,11,50
4867,18,2022,12,50


# Merge Crime Fact Table with dimensions

In [21]:
crimedf = crimedf.merge(premisedf, on=['location_type','premises_type'])
crimedf = crimedf.drop(['location_type','premises_type'], axis=1)
crimedf = crimedf.merge(incidentdf, on=['mci_category','offence','ucr_code', 'ucr_ext'])
crimedf = crimedf.drop(['mci_category','offence','ucr_code', 'ucr_ext'], axis=1)
crimedf = crimedf.merge(locationdf, on=['Division', 'Longitude', 'Latitude'])
crimedf = crimedf.drop(['Division', 'Longitude', 'Latitude'], axis=1)
crimedf = crimedf.merge(occurencedf, on=['occurrencedate'])
crimedf = crimedf.drop(['occurrencedate','occurrencehour','occurrenceday','occurrencemonth','occurrenceyear'], axis=1)
crimedf = crimedf.merge(reporteddf, on=['reporteddate'])
crimedf = crimedf.drop(['reporteddate','reportedhour','reportedday','reportedmonth','reportedyear'], axis=1)
crimedf = crimedf.merge(neighbourhooddf, on=['Neighbourhood'])
crimedf = crimedf.drop(['Neighbourhood','Shape__Area','Shape__Length'], axis=1)
crimedf

,premise_PK,incident_PK,location_PK,occurence_PK,reported_PK,neighbourhood_PK
0,1,1,1,1,1,2
1,4,2,2455,1570,1239,2
2,8,2,4498,1570,1239,2
3,4,9,746,57,23,2
4,1,1,580,47,11,2
...,...,...,...,...,...,...
272093,1,1,13876,3404,2972,104
272094,13,1,1239,727,446,104
272095,21,1,1421,727,446,104
272096,6,10,5347,717,437,104


# Save Tables as CSV

In [ ]:
# Saving tables to csv
crimedf.to_csv('CrimeFactTable.csv', index=False)
casesdf.to_csv('CovidFactTable.csv', index=False)
neighbourhooddf.to_csv('NeighbourhoodTable.csv', index=False)
incidentdf.to_csv('IncidentTable.csv', index=False)
premisedf.to_csv('PremiseTable.csv', index=False)
locationdf.to_csv('LocationTable.csv', index=False)
occurencedf.to_csv('OccurrenceTable.csv', index=False)
reporteddf.to_csv('ReportedTable.csv', index=False)